In [1]:
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import folium
from folium.plugins import HeatMap, FastMarkerCluster
import dash_bootstrap_components as dbc
import re

/home/christos/anaconda3/lib/python3.8/site-packages/dash_bootstrap_components/table.py:1: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
external_stylesheets = [dbc.themes.LUX]

In [3]:
df= pd.read_csv('../data/Books_final(cleaned).csv')

In [4]:
df1= pd.read_csv('../data/for_dash.csv')

In [5]:
def value_filter_1(df,n):
    return df.iloc[0:n,:]

def value_filter_2(df,year):
    return df[(df.Year == year)]

def value_filter_3(df, value):
    fp = pd.DataFrame(df.Author.value_counts())
    return fp.head(value).sort_values(by='Author',ascending=True)

In [6]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
 
#FIRST ROW----------------------(HEADER)-------------------------------------------------------  
    dbc.Row([ 
       dbc.Col(html.H1('Captainbook.gr/ Bookstore Analysis', 
                        className = 'text-center text-primary mb-4'),
                width=12)
    ]),

#SECOND ROW------------------------------------------------------------------------------------
    dbc.Row([      
        dbc.Col([html.Label(['Select Number of Pairs to Showcase:'], style={'font-weight': 'bold',"textDecoration": "underline", 
                                        'text-align':'center'}),
            dcc.Slider(
                id='my-slider',
                min=0,
                max=27,
                step=1,
                value=10,
            ), 
            html.Div(id='slider-output-container'),
                html.Br(),
        
        
                dcc.Graph(id="pairs", figure={}),                              
                html.Br(),
                ]),
    ]),

#THIRD ROW----------------------------------------------------------------------        
    dbc.Row([           
        dbc.Col([html.Label('Select Year:', style={'font-weight': 'bold',"textDecoration": "underline"}, 
                            className ='text-center'),
        dcc.Dropdown(
                id='dropdown',
                options=[
                    {'label': '2016', 'value': 2016},
                    {'label': '2017', 'value': 2017},
                    {'label': '2018', 'value': 2018},
                    {'label': '2019', 'value': 2019},
                    {'label': '2020', 'value': 2020},
        
        ],
        value=2020,
        style={'width':"50%"}
    )
                ]),
    ]),
    html.Br(),

#FOURTH ROW----------------------------------------------------------------------       
     
    dbc.Row([        
        dbc.Col([dcc.Graph(id='scatter',figure={}),
        html.Br(),
                ]),
        
        dbc.Col([dcc.Graph(id="hist", figure={}) ,              
            html.Br()
                    ]),
          ]),

#FIFTH ROW----------------------------------------------------------------------      
    
    dbc.Row([           
            dbc.Col([html.Label('Select Number of Authors to Showcase:', style={'font-weight': 'bold',"textDecoration": "underline"}, 
                                className ='text-center'),
                dcc.RadioItems(id='radio',value=10, 
                              options = [{'label':10, 'value':10},
                                         {'label':15, 'value':15},
                                         {'label':20, 'value':20},
                                         {'label':25, 'value':25},
                                         {'label':30, 'value':30}],
                               labelStyle={'display': 'inline-block'},
                          labelClassName="mr-3")
                        
            
        ], width={'size':6, 'offset':3}),
    ]),
#SIXTH ROW------------------------------------------------------------------------------    
    dbc.Row([

            
        dbc.Col([dcc.Graph(id='barplot', figure={}),               
             ]),
    ]),
])
    


    
    
#callback for pairs------------------------------------------------------------   

@app.callback(
    [Output('pairs', component_property ='figure'),
    Output('slider-output-container', component_property ='children')],
    Input('my-slider', component_property ='value')
)
def update_graph(value):
    dff=df1
    fig = px.bar(value_filter_1(dff,value), x = 'Qty', y='Pair',labels={'Qty':'Quantity'}, 
                 color='Pair', height=600, 
                 title='Most Frequent Pair of Categories')   
    
    return fig, 'You have selected "{}"'.format(value)

#callback for scatter & hist----------------------------------------------------------------------

@app.callback(
    [Output("scatter", component_property ='figure'),
    Output('hist',component_property ='figure')],
     Input('dropdown', component_property ='value'),
)

def update_graph(value):
    dff = df
    
    fig1 = px.scatter(value_filter_2(dff,value), x="Price_New", y="Pages",
                 size='Pages', hover_data=['Title'], height=600,width=500, 
                 title = 'Price(€) in relation to the length of each book', 
                      labels ={'Price_New':'Price',"Pages":'Number of Pages'})
    
    
    df3 = value_filter_2(dff,value)
    
    fig2 = px.histogram(df3['Discount'], height=600, width=500, 
                        title='Discounts Distribution per Year')
    
    return fig1 , fig2

#callback for barplot----------------------------------------------------------------------

@app.callback(
    
    Output('barplot', component_property ='figure'),
    Input('radio', component_property ='value')
)
def update_graph(value):
    dff=df
    
    fig = px.bar(value_filter_3(dff,value), x = 'Author', y=value_filter_3(dff,value).index,  height=600, 
                 title='Authors with the most Book Recordings')   
    
    return fig



#run app------------------------------------------------------------------------------------

app.run_server(mode ='jupyterlab')

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/christos/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/christos/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/christos/anaconda3/lib/python3.8/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/home/christos/anaconda3/lib/python3.8/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/home/christos/anaconda3/lib/python3.8/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/home/christos/anaconda3/lib/python3.8/site-packages/six.py", line 703, in reraise
    raise value
  File "/home/christos/anaconda3/lib/python3.8/site-packages/retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
  File "/home/